# Computer Tomography

In this example, we use data from a *computer tomography* (CT) study of a cadaver head: <https://graphics.stanford.edu/data/voldata/>

In [1]:
import libcarna

Get the data:

In [2]:
data = libcarna.data.cthead()
data.shape, data.dtype

((256, 256, 99), dtype('uint16'))

The data is 256 × 256 × 99 pixels (uint16).

## Maximum Intensity Projection

In the code below, we use `normals=True` on the `volume` node; albeit this doesn't make any difference for the *Maximum
Intensity Projection* (MIP), it is benefitial for other rendering modes, [discussed below](#Direct-Volume-Rendering).

In [3]:
GEOMETRY_TYPE_VOLUME = 2

# Create and configure frame renderer
mip = libcarna.mip(GEOMETRY_TYPE_VOLUME, sr=400)
r = libcarna.renderer(600, 450, [mip])

# Create and configure scene
root = libcarna.node()
volume = libcarna.volume(
    GEOMETRY_TYPE_VOLUME,
    data / data.max(),
    parent=root,
    spacing=(1, 1, 2),
    normals=True,
).rotate('z', 90).rotate('x', 90)
camera = libcarna.camera(
    parent=root,
).frustum(fov=90, z_near=10, z_far=1000).translate(0, 0, 300)

# Render
libcarna.imshow(r.render(camera))

The spatial structure of the 3D image is difficult to perceive from that rendering.

For a better visual perception, viewing the data from different angles is benefical, that can be achieved with
as a subtle animation:

In [4]:
# Render as animation
libcarna.imshow(
    libcarna.animate(
        libcarna.animate.rotate_local(camera),
        n_frames=100,
    ).render(r, camera),
)

## Direct Volume Rendering

In a *Direct Volume Rendering* (DVR), surfaces are rendered by simulation of the absorption of light. This simulation
is most realstic, when the spatial orientation of the surfaces can be taken into account, which requires that the
normals of the volume have been computed (this is why we used `normals=True` when we created the `volume` node).

We use a quadratic ramp function to strip out the air from the visualization:

In [5]:
dvr = libcarna.dvr(
    GEOMETRY_TYPE_VOLUME, sr=800, transl=1, diffuse=0.8,
)
dvr.cmap.BrBG(ramp=0.5, rampdegree=2)

libcarna.imshow(
    libcarna.animate(
        libcarna.animate.rotate_local(camera),
        n_frames=100,
    ).render(
        libcarna.renderer(600, 450, [dvr]),
        camera,
    ),
)

## Hounsfield Unit Normalization

The data from CT scanners usually comes in *Hounsfield Units* (HU), that range from -1024 to +3071. In the HU scale,
air roughly corresponds to -1000 HU, water to 0 HU, and bone tissue to +1000 HU. The image intensities in this dataset
are not normalized to the HU scale. Normalization of CT data to the HU scale is beneficial, because it allows to
render *Digitally Reconstructed Radiographs* (DRR), as shown [further below](#Digitally-Reconstructed-Radiographs).

LibCarna-Python provides a heuristic method for *approximative* normalization of CT data to the HU scale, that is based
on the histogram of the intensisty values:

In [6]:
data_hu = libcarna.normalize_hounsfield_units(data)

## Digitally Reconstructed Radiographs

*Digitally Reconstructed Radiographs* (DRRs) are 2D images created from 3D data, like CT scans, to simulate what a real
X-ray image would look like.

To create a DRR, first define a volume with the HU-normalized image intensities:

In [7]:
GEOMETRY_TYPE_HU_VOLUME = 3

libcarna.volume(
    GEOMETRY_TYPE_HU_VOLUME,
    data_hu,
    units='hu',
    parent=root,
    spacing=volume.spacing,
    local_transform=volume.local_transform,
)

<libcarna._spatial.volume.<locals>.WrapperNode at 0x7ea6e1588410>

Note that `units='hu'` is needed to correctly interpret the intensities in `data_hu`.

Next, the DRR rendering can be performed:

In [8]:
libcarna.imshow(
    libcarna.animate(
        libcarna.animate.rotate_local(camera),
        n_frames=100,
    ).render(
        libcarna.renderer(
            600, 450, [
                libcarna.drr(
                    GEOMETRY_TYPE_HU_VOLUME, sr=800, inverse=True,
                )
            ],
            bgcolor=libcarna.color.WHITE_NO_ALPHA,
        ),
        camera,
    ),
)